In [2]:
import json
import ast
import gzip
import pandas as pd

#Abro el archivo de User REVIEWS
data = []
with gzip.open('Archivos_Originales/user_reviews.json.gz','rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode("latin-1")))
reviews=pd.DataFrame(data)
f.close()

# Configura pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [3]:
#Voy a ver que hay en la columna "reviews"
reviews.reviews.iloc[5]

[{'funny': '',
  'posted': 'Posted May 5, 2014.',
  'last_edited': '',
  'item_id': '249130',
  'helpful': '7 of 8 people (88%) found this review helpful',
  'recommend': True,
  'review': 'This game is Marvellous.'},
 {'funny': '',
  'posted': 'Posted December 24, 2012.',
  'last_edited': 'Last edited November 25, 2013.',
  'item_id': '207610',
  'helpful': '1 of 1 people (100%) found this review helpful',
  'recommend': True,
  'review': 'It reminds me of that TV Show called "The Walking Dead".'},
 {'funny': '1 person found this review funny',
  'posted': 'Posted October 21, 2012.',
  'last_edited': 'Last edited November 25, 2013.',
  'item_id': '550',
  'helpful': '1 of 1 people (100%) found this review helpful',
  'recommend': True,
  'review': 'This game is fantastic if you are looking to DEADicate some time to it.'},
 {'funny': '',
  'posted': 'Posted March 20, 2012.',
  'last_edited': 'Last edited June 22, 2014.',
  'item_id': '65800',
  'helpful': '1 of 1 people (100%) found th

In [4]:
#Desanidando el diccionario que está en reviews. 
#Genero una review por usuario (antes tenia todas juntas por usuario)

final_reviews = reviews.explode('reviews') # Genero el df separado por review > en cada fila voy a tener diferentes reviews que pueden pertenecer a  un mismo usuario
final_reviews.head(5)
final_reviews.reviews.iloc[0]

{'funny': '',
 'posted': 'Posted November 5, 2011.',
 'last_edited': '',
 'item_id': '1250',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}

In [5]:
#Reseteo el index
final_reviews.index = range(len(final_reviews)) # genero un indice distinto por cada reviews ( y no por usuario)

final_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [6]:
#Voy a desanidar cada review
#Creo un df a partir de la lista de elementos anidados en la review.

reviews_desanidados = pd.json_normalize(final_reviews.reviews) # Le paso la columna de reviews. Genera una lista d elementos anidados. Armo un DF, con cada clave del diccionario dentro de reviews como una columna
reviews_desanidados.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [7]:
#Asocio cada review con el user id correspondiente: 
reviews_clean = pd.concat([final_reviews.drop(columns= ['reviews']), reviews_desanidados], axis=1) #concateno el df extendido (con una fila por review de usuario) con el df con una columna por key de review

In [8]:
reviews_clean.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [9]:
#Veo info del df
reviews_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


In [10]:
#Dimensiones del dataframe: cant. de filas y cant. de  columnas
print(f"Dimensiones del dataframe de reviews: {reviews_clean.shape}") 

Dimensiones del dataframe de reviews: (59333, 9)


In [11]:
#Columnas del data frame en una lista
print(reviews_clean.columns)

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'review'],
      dtype='object')


In [12]:
#Cant. de nulos por columna: numero total de faltantes por columna
print(f"La cantidad de nulos para cada columna es: \n{reviews_clean.isna().sum()}") 


La cantidad de nulos para cada columna es: 
user_id         0
user_url        0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64


In [13]:
#Promedio de la cantidad de nulos por columna
#Vemos que practicamente no hay nulos, por lo que el dataset esta casi completo.
print(f"El promedio de la cantidad de nulos es: \n{reviews_clean.isna().mean()}") 

El promedio de la cantidad de nulos es: 
user_id        0.000000
user_url       0.000000
funny          0.000472
posted         0.000472
last_edited    0.000472
item_id        0.000472
helpful        0.000472
recommend      0.000472
review         0.000472
dtype: float64


In [14]:
#Veo las ultimas 5 filas del file de reviews
reviews_clean.tail(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D
59332,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 2.,,440,No ratings yet,True,so much fun :D


In [15]:
#Voy a eliminar las filas con nulos.
#Con la especificación how="all" voy a indicarle que elimine las filas donde TODOS los elementos son nulos.
reviews_clean=reviews_clean.dropna(how="all")

print(reviews_clean.shape)

(59333, 9)


In [16]:
#Voy a eliminar las reviews que estan duplicadas
duplicados= reviews_clean.duplicated()

#print(sum(duplicados)) #Devuelve 874, asique voy a eliminar esos duplicados.

reviews_sinduplicados=reviews_clean.drop_duplicates()

print(reviews_sinduplicados.shape)

(58459, 9)


In [17]:
reviews_sinduplicados.head(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [18]:
#Elimino columnas que no voy a necesitar y me quedo un df mas corto.

reviews_final= reviews_sinduplicados.drop(columns=["user_url","funny","last_edited","helpful","recommend"])

reviews_final.head(5)

,user_id,posted,item_id,review
0,76561197970982479,"Posted November 5, 2011.",1250,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,For a simple (it's actually not all that simpl...


In [19]:
print(f"Dimensiones del dataframe de games: {reviews_final.shape}") 

Dimensiones del dataframe de games: (58459, 4)


In [20]:
#Como voy a necesitar la fecha, en la columna de posted voy a eliminar los strings y quedarme solo con la fecha.
reviews_final["posted"] = reviews_final.posted.str.replace('Posted', '')

reviews_final.head(2)

,user_id,posted,item_id,review
0,76561197970982479,"November 5, 2011.",1250,Simple yet with great replayability. In my opi...
1,76561197970982479,"July 15, 2011.",22200,It's unique and worth a playthrough.


In [21]:
#Dado que solamente necesito el año, voy a transformar la columna de fecha en el año correspondiente:
reviews_final["posted"] = pd.to_datetime(reviews_final.posted, format='mixed', errors='coerce').dt.year


reviews_final.head(2)

,user_id,posted,item_id,review
0,76561197970982479,2011.0,1250,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,It's unique and worth a playthrough.


In [22]:
#Paso el archivo a un parquet
reviews_final.to_parquet("Archivos_Comprimidos/reviews_final.parquet",index=False)

reviews_final.head(2)

,user_id,posted,item_id,review
0,76561197970982479,2011.0,1250,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,It's unique and worth a playthrough.


In [23]:
reviews_prueba= pd.read_parquet("Archivos_Comprimidos/reviews_final.parquet")

reviews_prueba.head(2)

,user_id,posted,item_id,review
0,76561197970982479,2011.0,1250,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,It's unique and worth a playthrough.


In [24]:
reviews_prueba.head(5)


,user_id,posted,item_id,review
0,76561197970982479,2011.0,1250,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,It's unique and worth a playthrough.
2,76561197970982479,2011.0,43110,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014.0,251610,I know what you think when you see this title ...
4,js41637,2013.0,227300,For a simple (it's actually not all that simpl...
